# Configurations
This is where all configs will be set before the script makes an API call [via openmeteo] to collect weather data.

In [1]:
import collections

In [2]:
locations = {

    "22.258351741017634, 114.21163275365531" : "Tai Tam Reservoir" ,
    "22.27054210988102, 114.00032030347596"  : "Mui Wo",
    "22.540797536909047, 114.43437970480949" : "Tung Ping Chau",
    "22.29770991812577, 114.30310328185212" : "Clear Water Bay",
    "22.239390991398583, 114.24173429924463" : "Dragon's Back",
    "22.23398579735171, 114.15930196830752" : "Ap Lei Chau",
    "22.28617189441449, 114.04065619628331" : "Peng Chau"

}

locations = collections.OrderedDict(sorted(locations.items()))

In [10]:
MAX_WIND_kph     = 35
MAX_PRECIP_mm    = 0
MAX_CLOUDS_pcnt  = 50

In [3]:
VERBOSE_MODE = True

# Processing of raw data

In [4]:
from urllib.request import urlopen, Request
import json
import pandas as pd
import datetime
import pytz

In [5]:
def get_flytimes(latlong, name, maxwind_kph, maxprecip_mm, maxclouds_pcnt, is_verbose):
  # Construct URL
  gps_loc = latlong.split(",")
  latitude = str(round(float(gps_loc[0].strip()),4))
  longitude = str(round(float(gps_loc[1].strip()),4))
  url = 'https://api.open-meteo.com/v1/forecast?latitude=' + latitude + '&longitude=' \
    + longitude +'&hourly=precipitation,cloudcover,windgusts_10m&timezone=auto'

  if is_verbose:
    print(url)
    print("Weather Forecast: {} \nGPS: {}".format(name, latlong))

  # Retrieve data
  request = Request(url)
  response = urlopen(request)
  forecast = response.read()
  data = json.loads(forecast)
  df_json = pd.json_normalize(data)

  list_gust = df_json["hourly.windgusts_10m"][0]
  list_time = df_json["hourly.time"][0]
  list_precip = df_json["hourly.precipitation"][0]
  list_cloud = df_json["hourly.cloudcover"][0]

  # Assert that length of lists are equal to ensure count of rows are equal
  assert len(list_gust) == len(list_time) and len(list_time) == len(list_precip) and len(list_precip) == len(list_cloud)

  # Init. custom DF
  timezone = df_json['timezone'][0]
  col_time = "Datetime (" + timezone + ")"
  col_gust = "Gust (" + df_json['hourly_units.windgusts_10m'][0] + ")"
  col_precip = "Precip. (" + df_json['hourly_units.precipitation'][0] + ")"
  col_cloud = "Cloud Cover (" + df_json['hourly_units.cloudcover'][0] + ")"

  # Filter: Only include daytime
  tz = df_json["timezone"][0]
  datetimenow = datetime.datetime.now(pytz.timezone(tz))
  datefilter = []
  list_dayofweek = []
  # Mask: is_daytime = hour between 6am and 6pm
  for i in range(len(list_time)):
    substr1   = list_time[i].split('-')
    yyyy, mm  = int(substr1[0]), int(substr1[1])
    substr2     = substr1[2].split('T')
    dd, hh      = int(substr2[0]), int(substr2[1].split(':')[0])
    is_daytime    = hh < 18 and hh > 6
    datefilter.append(dd >= datetimenow.day and mm >= datetimenow.month and yyyy >= datetimenow.year and is_daytime)
    str_dayofweek = datetime.date(yyyy,mm,dd).strftime('%A')
    list_dayofweek.append(str_dayofweek)
    list_time[i] = list_time[i]+ " " + str_dayofweek.ljust(10, ' ') 

  df_custom = pd.DataFrame({col_time : list_time,
    col_gust : list_gust,
    col_precip : list_precip,
    col_cloud : list_cloud
  })

  # df_custom["Day"] = list_dayofweek
  df_custom["Date Filter"] = datefilter
  
  # Filter: Gust & Precip. & Datetime
  _condition1 = df_custom[col_gust] <= maxwind_kph
  _condition2 = df_custom[col_precip] <= maxprecip_mm
  _condition3 = df_custom[col_cloud] <= maxclouds_pcnt
  _condition4 = df_custom["Date Filter"]

  # Apply the conditions and drop unnecessary columns
  df_custom_filtered = df_custom[_condition1 & _condition2 & _condition3 & _condition4]
  df_custom_filtered = df_custom_filtered.drop(columns=["Date Filter", col_precip])
  
  # Report by location
  if is_verbose:
    print("Report generated on: " + datetimenow.strftime("%A %d. %B %Y"))
    print(df_custom_filtered.to_string(index=False, justify='left'))
    print('\n')

  return df_custom_filtered[col_time]
  

# Output 


In [6]:
detailed_report = {} # key: datetime, value: list of locations that are suitable for flying
location_names = [] # headers for the final report

for key in locations:
  name = locations[key].split(";")

  if not name[0] in location_names:
    location_names.append(name[0])
  
  _flytimes = get_flytimes(key, name[0], MAX_WIND_kph, MAX_PRECIP_mm, MAX_CLOUDS_pcnt, is_verbose=VERBOSE_MODE) 

  for str_datetime in _flytimes:
    if not str_datetime in detailed_report:
      detailed_report[str_datetime] = []
    detailed_report[str_datetime].append(name[0])

detailed_report = collections.OrderedDict(sorted(detailed_report.items()))

https://api.open-meteo.com/v1/forecast?latitude=22.234&longitude=114.1593&hourly=precipitation,cloudcover,windgusts_10m&timezone=auto
Weather Forecast: Ap Lei Chau 
GPS: 22.23398579735171, 114.15930196830752
Report generated on: Tuesday 16. January 2024
Datetime (Asia/Hong_Kong)    Gust (km/h)  Cloud Cover (%)
2024-01-17T08:00 Wednesday  30.2         49              
2024-01-18T15:00 Thursday   28.4         42              
2024-01-18T16:00 Thursday   25.9         19              
2024-01-18T17:00 Thursday   23.4          0              
2024-01-19T07:00 Friday     12.2          0              
2024-01-19T08:00 Friday      8.3          0              
2024-01-19T09:00 Friday     12.2         15              
2024-01-19T10:00 Friday     16.6         30              
2024-01-19T11:00 Friday     20.5         45              
2024-01-20T07:00 Saturday   10.8         16              
2024-01-20T08:00 Saturday   10.8         24              
2024-01-20T09:00 Saturday   11.5         29       

In [7]:
summary_report = {} # key: location, value: list of ✓ or " " for each datetime

for str_time in detailed_report:
  for str_name in location_names:
    if not str_name in summary_report:
      summary_report[str_name] = []
    if str_name in detailed_report[str_time]:
      summary_report[str_name].append("✓")
    else:
      summary_report[str_name].append(" ")

df_summary_report = pd.DataFrame(summary_report).assign(time=list(detailed_report.keys()))
cols = df_summary_report.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_summary_report = df_summary_report[cols]

In [8]:
def highlight_max(s):
    is_max = s == s.max() 
    return ['color: #000000; background-color: #FFFFFF' if v else '' for v in is_max]

df_formatted = df_summary_report.style\
  .apply(highlight_max, subset=location_names)\
  .set_table_styles([dict(selector="th",props=[('max-width', '50px')])])\
  .set_properties(**{'text-align': 'left'})\
  .hide_index()

C:\Users\kevli\AppData\Local\Temp\ipykernel_25412\2342590604.py:9: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  .hide_index()


In [9]:
df_formatted

time,Ap Lei Chau,Dragon's Back,Tai Tam Reservoir,Mui Wo,Peng Chau,Clear Water Bay,Tung Ping Chau
2024-01-16T16:00 Tuesday,,,,,,,✓
2024-01-16T17:00 Tuesday,,,,,,,✓
2024-01-17T07:00 Wednesday,,,,,,,✓
2024-01-17T08:00 Wednesday,✓,✓,✓,✓,✓,✓,✓
2024-01-17T09:00 Wednesday,,✓,✓,,,✓,✓
2024-01-17T10:00 Wednesday,,,,,,,✓
2024-01-17T17:00 Wednesday,,,,,,,✓
2024-01-18T09:00 Thursday,,,,,,,✓
2024-01-18T10:00 Thursday,,,,,,,✓
2024-01-18T11:00 Thursday,,,,,,,✓
